# Metrics - CSAT (for ABSA, not topic modelling)

In [1]:
import duckdb as ddb
import sys
from pathlib import Path
common_code_path = Path.cwd().parent.parent / "plp_common_code"
print(common_code_path)

if str(common_code_path) not in sys.path:
    sys.path.append(str(common_code_path))
from amhere import foundme
foundme()  # this fn will fail if common code path is not found or !exists
del foundme
del common_code_path

cwd = Path.cwd()
db_path = cwd / 'database'
db_file_name = 'aspect_topic_db.parquet'  # db path
database_path_file = db_path / db_file_name
# self._output_path = cwd / 'data_output'
print(database_path_file)

D:\OneDrive\RChai\Documents\NUS_ISS_Prjs\4_PLP\06_Code_Actuals\plp_common_code
D:\OneDrive\RChai\Documents\NUS_ISS_Prjs\4_PLP\06_Code_Actuals\3.4_Components_Deployed_Soln\3.4.8_Cust_Metrics\database\aspect_topic_db.parquet


In [2]:
df = ddb.read_parquet(str(database_path_file)).to_df()
df.columns

Index(['date', 'dlg_id', 'text', 'product', 'action', 'sentiment',
       'aspect_topic'],
      dtype='object')

In [3]:
# dft = ddb.query("""select count(*) as num_rows from df;""").to_df()
# dft

# Get subset data by date rang

In [4]:
start_date = '2024-10-05'
end_date = '2024-10-05'

sql_query = f"""
select * from df where date >= '{start_date}' and date <= '{end_date}';
"""
dft = ddb.query(sql_query).to_df()
display(dft.head(3))
display(dft.tail(3))

date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   

                                                text product   action  \
0                      i am still waiting on my card    card  inquire   
1  what can i do if my card still hasnt arrived a...    card  inquire   
2  i have been waiting over a week is the card st...    card  inquire   

  sentiment   aspect_topic  
0   neutral  inquire__card  
1   neutral  inquire__card  
2   neutral  inquire__card

date               dlg_id  \
13971  2024-10-05  2024-10-14__dlg_992   
13972  2024-10-05  2024-10-14__dlg_992   
13973  2024-10-05  2024-10-14__dlg_992   

                                                    text     product   action  \
13971  Hi Alex, I hope you're doing well. I'm very im...        loan    block   
13972  Sure, the website is [Website Name]. While the...        loan  dispute   
13973  That would be perfect. Please go ahead and set...  debit_card   access   

           sentiment        aspect_topic  
13971  very positive         block__loan  
13972  very negative       dispute__loan  
13973  very negative  access__debit_card

# CSAT

In [5]:
# def convert_sentiment_to_csat_labels(x):
#     x = x.lower()
#     match x:
#         case 'very negative':  # aka Very Unsatisfied
#             return 'very unsatisfied'
#         case 'negative':  # Unsatisfied
#             return 'unsatisfied'
#         case 'neutral':
#             return 'neutral'
#         case 'positive':  # Satisfied
#             return 'satisfied'
#         case 'very positive':  # Very Satisfied
#             return 'very satisfied'
#         case _:
#             raise SystemError('sentiment polarity is unknown value')
    
# dft['csat_group'] = dft['sentiment'].apply(lambda x: convert_sentiment_to_csat_labels(x))
# dft[['sentiment', 'csat_group']].sample(5)

In [6]:
def convert_sentiment_to_csat_lbl_score(x):
    match x:
        case 'very negative':  # aka Very Unsatisfied
            return 1
        case 'negative':  # Unsatisfied
            return 2
        case 'neutral':
            return 3
        case 'positive':  # Satisfied
            return 4
        case 'very positive':  # Very Satisfied
            return 5
        case _:
            raise SystemError('sentiment polarity is unknown value')
    
dft['csat_lbl_score'] = dft['sentiment'].apply(lambda x: convert_sentiment_to_csat_lbl_score(x))
dft[['aspect_topic', 'sentiment', 'csat_lbl_score']].sample(5)

aspect_topic      sentiment  csat_lbl_score
10303      redeem__forex_account  very positive               5
12053             activate__loan       positive               4
1729               inquire__card       positive               4
8555   retrieve__savings_account        neutral               3
13073   unknown__savings_account  very negative               1

### Calculate CSAT Groups for Bank

In [7]:
# not completed yet

sql_query = f"""
select 
aspect_topic 
,avg(csat_lbl_score) as avg_csat_lbl_score 
,case
    when avg_csat_lbl_score >= 4.5 then 'very satisfied'
    when avg_csat_lbl_score >= 3.1 then 'satisfied'
    when avg_csat_lbl_score >= 2.9 then 'neutral'
    when avg_csat_lbl_score >= 1.5 then 'unsatisfied'
    else 'very unsatisfied'
end as csat_group 
, ANY_VALUE(dlg_id) as rep_dlg_id -- representative dialog id
, ANY_VALUE(text) as rep_text  -- representative text
from dft
group by aspect_topic
order by avg_csat_lbl_score desc, aspect_topic;
"""
df_csat_groups = ddb.query(sql_query).to_df()
df_csat_groups.sample(10)

aspect_topic  avg_csat_lbl_score   csat_group  \
266     reset__mobile_app            2.970588      neutral   
227         unblock__loan            3.066667      neutral   
222         dispute__card            3.083019      neutral   
65          deposit__card            3.601449    satisfied   
12        select__website            4.166667    satisfied   
299  activate__debit_card            2.847826  unsatisfied   
232    update__debit_card            3.055556      neutral   
325          refund__loan            2.750000  unsatisfied   
238     block__debit_card            3.037736      neutral   
373          report__loan            2.400000  unsatisfied   

                rep_dlg_id                                           rep_text  
266   2024-10-07_1017--844                can i use app to reset pin attempts  
227    2024-10-14__dlg_495  It was blocked just yesterday when I was tryin...  
222   2024-10-07_1017--409  the exchange rate from my card payment isnt right  
65    2024-10-07_1017--770  can i add money to my card automatically durin...  
12     2024-10-14__dlg_160  That sounds exactly like what I need! I really...  
299    2024-10-14__dlg_171  Yes, I understand. I’ve considered those point...  
232   2024-10-14__dlg_1145  That sounds perfect, Alex. I really appreciate...  
325   2024-10-14__dlg_1980  Sure, the loan account number is 123456789. I ...  
238  2024-10-07_1017--6872  freeze my debit card i want to stop at all tra...  
373   2024-10-14__dlg_1051  I shouldn't have to deal with this stress! The...

### Calc CSAT

In [8]:
sql_query = f"""
with satisfied as (
    select count(*) as num_satisfied from df_csat_groups where csat_group = 'satisfied' or csat_group = 'very satisfied' 
)
,responses as (select count(*) as total_responses from df_csat_groups)
select s.num_satisfied, r.total_responses
,(s.num_satisfied/r.total_responses * 100) as csat_score
from satisfied as s, responses as r;
"""
df_csat = ddb.query(sql_query).to_df()
df_csat

num_satisfied  total_responses  csat_score
0            218              393   55.470738